<a href="https://colab.research.google.com/github/NevesJulio/Exercicios_DL/blob/main/Atividade_DeepLearning_Labubu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade: CNNs para Classificação

Neste notebook, iremos preparar nosso próprio dataset e treinar um modelo de classificação de imagens.

## Preparando os dados

Os dados desta atividade serão baixados da internet. Utilizaremos para isso buscadores comuns. Em seguida, dividiremos em treinamento e validação.

In [1]:
%pip install icrawler

In [2]:
import os
import shutil
import random
from icrawler.builtin import GoogleImageCrawler, BingImageCrawler

### Adquirindo as Imagens

Utilizaremos o iCrawler para baixar imagens em buscadores através de termos especificados. Defina sua lista de classes.

In [3]:
def download_images(keyword, folder, n_total=100):
    os.makedirs(folder, exist_ok=True)
    downloaded = len(os.listdir(folder))
    remaining = n_total - downloaded

    while downloaded < n_total:
        crawler = GoogleImageCrawler(storage={'root_dir': folder})
        crawler.crawl(keyword=keyword, max_num=remaining, file_idx_offset=downloaded)
        downloaded = len(os.listdir(folder))
        remaining = n_total - downloaded
        print(f"Downloaded {downloaded}/{n_total}")

    print("Download complete!")

In [4]:
search_terms = {
     "Labubu": "Labubu", # nome da classe: termo que será usado na busca
     "Pikachu": "pikachu",
     "sagui": "sagui"

}

for label, term in search_terms.items():
    download_images(term, f"data/Julio/{label}", n_total=100)

ERROR:downloader:Exception caught when downloading file https://cdn-images.farfetch-contents.com/31/78/64/07/31786407_61234927_600.jpg, error: HTTPSConnectionPool(host='cdn-images.farfetch-contents.com', port=443): Read timed out. (read timeout=5), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://cdn-images.farfetch-contents.com/31/78/64/07/31786407_61234927_600.jpg, error: HTTPSConnectionPool(host='cdn-images.farfetch-contents.com', port=443): Read timed out. (read timeout=5), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://cdn-images.farfetch-contents.com/31/78/64/07/31786407_61234927_600.jpg, error: HTTPSConnectionPool(host='cdn-images.farfetch-contents.com', port=443): Read timed out. (read timeout=5), remaining retry times: 0
ERROR:downloader:Response status code 401, file https://i.guim.co.uk/img/media/2a75b01d900399e8c42de70ec72c05e4787b7092/301_0_2501_2001/master/2501.jpg
ERROR:downloader:Response 

Downloaded 63/100
Downloaded 64/100


ERROR:downloader:Exception caught when downloading file https://cdn-images.farfetch-contents.com/31/78/64/07/31786407_61234927_600.jpg, error: HTTPSConnectionPool(host='cdn-images.farfetch-contents.com', port=443): Read timed out. (read timeout=5), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://cdn-images.farfetch-contents.com/31/78/64/07/31786407_61234927_600.jpg, error: HTTPSConnectionPool(host='cdn-images.farfetch-contents.com', port=443): Read timed out. (read timeout=5), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://cdn-images.farfetch-contents.com/31/78/64/07/31786407_61234927_600.jpg, error: HTTPSConnectionPool(host='cdn-images.farfetch-contents.com', port=443): Read timed out. (read timeout=5), remaining retry times: 0


Downloaded 100/100
Download complete!


ERROR:downloader:Response status code 403, file https://archives.bulbagarden.net/media/upload/thumb/4/49/Ash_Pikachu.png
ERROR:downloader:Response status code 403, file https://external-preview.redd.it/AE6Jw9dqaSdzrz_ZWl_IrwwSd5ViFyiA_sKMELQ9YPs.png
ERROR:downloader:Exception caught when downloading file https://www.png, error: HTTPSConnectionPool(host='www.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7a089021df10>: Failed to resolve 'www.png' ([Errno -2] Name or service not known)")), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://www.png, error: HTTPSConnectionPool(host='www.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7a087fc252b0>: Failed to resolve 'www.png' ([Errno -2] Name or service not known)")), remaining retry times: 1
ERROR:downloader:Exception caught when downloadi

...

Downloaded 100/100
Download complete!


ERROR:downloader:Response status code 403, file https://www.petz.com.br/blog/wp-content/uploads/2025/04/macaco-sagui-4-scaled.jpg
ERROR:downloader:Response status code 404, file https://t3.ftcdn.net/jpg
ERROR:downloader:Response status code 400, file https://media.istockphoto.com/id/1472424290/photo/sagui.jpg
ERROR:downloader:Response status code 429, file https://live.staticflickr.com/1560/26191448156_b3d285ddfd_b.jpg
ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/1367935230/photo/marmosets-sagui-monkey.jpg
ERROR:downloader:Response status code 429, file https://live.staticflickr.com/1918/44299593494_0f88a0918e_b.jpg
ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/975257608/photo/sagui.jpg
ERROR:downloader:Response status code 400, file https://media.istockphoto.com/id/164943444/photo/sagui-dwarf-monkey.jpg
ERROR:downloader:Response status code 429, file https://live.staticflickr.com/4750/26475814308_cf95905908_b.jpg
E

Downloaded 52/100


ERROR:downloader:Response status code 403, file https://www.petz.com.br/blog/wp-content/uploads/2025/04/macaco-sagui-4-scaled.jpg
ERROR:downloader:Response status code 404, file https://t3.ftcdn.net/jpg
ERROR:downloader:Response status code 429, file https://live.staticflickr.com/1560/26191448156_b3d285ddfd_b.jpg
ERROR:downloader:Response status code 403, file https://mid-educacao.curitiba.pr.gov.br/2022/8/jpg
ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/1367935230/photo/marmosets-sagui-monkey.jpg
ERROR:downloader:Response status code 429, file https://live.staticflickr.com/1918/44299593494_0f88a0918e_b.jpg
ERROR:downloader:Response status code 400, file https://www.ucs.br/zoo/_next/image?url=https%3A%2F%2Fsou.ucs.br%2Fapi%2Fzoo%2Fuploads%2Fimagens%2Fsagui-de-tufo-branco.png
ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/975257608/photo/sagui.jpg
ERROR:downloader:Response status code 429, file https://live.staticflic

Downloaded 100/100
Download complete!


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import copy
from PIL import Image
import glob

In [6]:
from torchvision import datasets, transforms, models


### Treinamento e Validação

Dividiremos as imagens baixadas nas pastas `train` e `val`. Defina uma porcentagem.

In [7]:
def split_train_val(root_dir, train_ratio=0.8, seed=42):
    random.seed(seed)

    train_dir = root_dir + "_split/train"
    val_dir = root_dir + "_split/val"

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    for class_name in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        images = [os.path.join(class_path, f) for f in os.listdir(class_path)]
        images = [f for f in images if os.path.isfile(f)]
        random.shuffle(images)

        n_train = int(len(images) * train_ratio)

        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        for img in images[:n_train]:
            shutil.copy(img, os.path.join(train_class_dir, os.path.basename(img)))
        for img in images[n_train:]:
            shutil.copy(img, os.path.join(val_class_dir, os.path.basename(img)))

        print(f"{class_name}: {n_train} train, {len(images)-n_train} val")

In [8]:
split_train_val("data/Julio")

sagui: 80 train, 20 val
Pikachu: 80 train, 20 val
Labubu: 80 train, 20 val


## Dataset

Implemente um Dataset PyTorch que carregue as imagens baixadas com suas respectivas classes. Aplique data augmentation e carregue em batches.

In [9]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.class_names = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.class_names)}

        self.data = []
        for class_name in self.class_names:
            class_dir = os.path.join(self.root_dir, class_name)
            for filename in os.listdir(class_dir):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    path = os.path.join(class_dir, filename)
                    item = (path, self.class_to_idx[class_name])
                    self.data.append(item)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path, label = self.data[idx]
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [15]:
DATA_DIR = "Data/DataSet"
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4
DEVICE = "cuda"
MODEL_NAME = "resnet50"
n_runs = 1
histories = {}
num_classes = 2
DROPOUT_RATE = 0.4


train_dataset = CustomImageDataset("data/Julio_split/train")
val_dataset   = CustomImageDataset("data/Julio_split/val")


In [11]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1, pin_memory=True)

In [14]:

# # Obtém um batch de dados de treino
#inputs, classes = next(iter(train_dataloader))

# # Cria uma grade a partir do batch
# out = utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

## Definição do Modelo

Defina aqui o modelo que será utilizado, sendo implementação própria ou um modelo pré-treinado.

In [21]:
class CustomHead(nn.Module):
    def __init__(self, in_features, num_classes, dropout_p=0.5):
        super().__init__()
        self.in_features = in_features
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Dropout(dropout_p),
            nn.Linear(in_features, 512),
            nn.ReLU(),

            nn.BatchNorm1d(512),
            nn.Dropout(dropout_p),
            nn.Linear(512, 128),
            nn.ReLU(),

            nn.BatchNorm1d(128),
            nn.Dropout(dropout_p),
            nn.Linear(128, 32),
            nn.ReLU(),

            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

In [19]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.fc = CustomHead(model.fc.in_features, 3, DROPOUT_RATE)
model = model.to(DEVICE)

In [22]:

for name, param in model.named_parameters():
    if "layer3" not in name and "layer4" not in name:
        param.requires_grad = False

model.fc = CustomHead(model.fc.in_features, num_classes, DROPOUT_RATE)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
filter(lambda p: p.requires_grad, model.parameters()),
lr=1e-4,
weight_decay=0.02  # L2 regularization
)


for i in range(n_runs):
    print(f"\n===== Run {i+1}/{n_runs} =====")

    scheduler = OneCycleLR(optimizer, max_lr=LR, steps_per_epoch=len(train_loader), epochs=EPOCHS)

    # Treina e retorna histórico
    history = train_and_validate(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=DEVICE,
        epochs=EPOCHS,
        save_path=f"{MODEL_NAME}_run{i}.pth",
        num_classes=num_classes
    )

    histories[f"run_{i}"] = history

    # Salva histórico em .pt
    torch.save(history, f"history_{MODEL_NAME}_run{i}.pt")

    # Salva histórico em CSV
    df = pd.DataFrame(history)
    csv_path = f"{MODEL_NAME}_run{i}_history.csv"
    df.to_csv(csv_path, index=False)
    print(f"Histórico salvo em CSV em {csv_path}")

    # Plota e salva gráficos
    plot_history(history, model_name=f"{MODEL_NAME}_run{i}")


AttributeError: 'CustomHead' object has no attribute 'in_features'

## Treinamento

Defina a função de custo e o otimizador do modelo. Em seguida, implemente o código de treinamento e treine-o. Ao final, exiba as curvas de treinamento e validação para a loss e a acurácia.

In [ ]:
# Seu código aqui

## Inferência

Calcule algumas métricas como acurácia, matriz de confusão, etc. Em seguida, teste o modelo em novas imagens das classes correspondentes mas de outras fontes (outro buscador, fotos próprias, etc).

In [ ]:
# Seu código aqui